In [16]:
import numpy as np
import pandas as pd
from collections import Counter 
import math
import pprint
data = pd.read_csv("weatherid3.csv")
df = pd.DataFrame(data)

attrs = list(df.columns)
print(attrs)
attrs.remove("Answer")
print(attrs)

['Outlook', 'Temperature', 'Humidity', 'Wind', 'Answer']
['Outlook', 'Temperature', 'Humidity', 'Wind']


In [10]:
def entr(a_list):
    cnt = Counter(a_list)
    probs = [inst/len(a_list) for inst in cnt.values()]
    entropy = sum([-prob*math.log(prob,2) for prob in probs])
    return entropy

print(entr(df["Answer"]))

0.9402859586706309


In [7]:
def propOb(a_list):
    return len(a_list)/len(df.index)

In [19]:
def info_gain(df,split_attr,tar_attr):
    df_split = df.groupby(split_attr)
    df_agg_ent = df_split.agg({tar_attr:[entr,propOb]})
    df_agg_ent.columns = ['Entr',"propOb"]
    newe = sum(df_agg_ent['Entr']*df_agg_ent['propOb'])
    olde = entr(df[tar_attr])
    return olde-newe

In [29]:
def id3(df,attrl,tar_attr):
    cnt = Counter(df[tar_attr])
    if len(cnt)==1:
        return next(iter(cnt))
    if df.empty or not attrl:
        return None
    
    gains = [info_gain(df,attr,tar_attr) for attr in attrl]
    iom = gains.index(max(gains))
    
    best = attrl[iom]
    tree = {best:{}} 
    
    rem = attrl.copy()
    rem.remove(best)
    
    for attr,ds in df.groupby(best):
        tree[best][attr] = id3(ds,rem,tar_attr)
        
    return tree

In [30]:
pprint.pprint(id3(df,attrs,'Answer'))

{'Outlook': {'overcast': 'yes',
             'rain': {'Wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}}}
